In [ ]:
## Version 13.09.21 - Global

In [ ]:
# -*- coding: utf-8 -*-

import sys
sys.path.append('./libs')

import os
os.environ["PROJ_LIB"] = r"~/anaconda3/Library/share"; #fixr

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import difflib
import sys
import matplotlib.pyplot as plt
import getMetadata as gmeta
import gc
import myGeoTools as mgt
import tracks_modules as tm
import pandas as pd
import time
import json

from fastdtw import fastdtw
from scipy import stats
from geopy.distance import geodesic as geodist
from similarityMeasures import Similarity
from datetime import date
from IPython.display import clear_output # progressbar

## Path folder settings

In [ ]:
EXP= r'./'

In [ ]:
path_in = EXP+r'entradas/'
file = 'out_tracks_Weddel.txt'
path_out = EXP+r'saidas/'

path_track = path_in + file

measures = Similarity()

## Oppening iceberg Dataset

In [ ]:
gc.collect()

detected_icebergs = pd.read_csv(path_track, sep=' ')

rows_to_drop = detected_icebergs.loc[detected_icebergs['date']=='date'].index

detected_icebergs.drop(rows_to_drop, inplace=True)

detected_icebergs['date'] = pd.to_datetime(detected_icebergs['date'].astype(str), format='%Y-%m-%d')
detected_icebergs['minoraxis_px'] = detected_icebergs['minoraxis_px'].astype('int')
detected_icebergs['majoraxis_px'] = detected_icebergs['majoraxis_px'].astype('int')
detected_icebergs['perimeter_px'] = detected_icebergs['perimeter_px'].astype('int')
detected_icebergs['area_px'] = detected_icebergs['area_px'].astype('int')

detected_icebergs['latitude'] = detected_icebergs['latitude'].astype('float')
detected_icebergs['longitude'] = detected_icebergs['longitude'].astype('float')

detected_icebergs.head()

## Convert pixel to Km and add to Dataframe

In [ ]:
gc.collect()

pixel_size = 75.
detected_icebergs['minoraxis_km'] = round((detected_icebergs['minoraxis_px'] * pixel_size) * 1e-3, 3)
detected_icebergs['majoraxis_km'] = round((detected_icebergs['majoraxis_px'] * pixel_size) * 1e-3, 3)
detected_icebergs['perimeter_km'] = round((detected_icebergs['perimeter_px'] * pixel_size) * 1e-3, 3)
detected_icebergs['area_km2'] = round((detected_icebergs['area_px'] * (pixel_size**2)) * 1e-6, 3)
#detected_icebergs['mass_gt'] = round((((detected_icebergs['area_km2'] * 1e6) * 250) * 850) * 1e-12, 3) 

#used_col = detected_icebergs.pop('used')
#detected_icebergs['used'] = used_col
n_total_detections = len(detected_icebergs['date'])
n_total_detections

## Filter by size

In [ ]:
min_size = 1.
max_size = 4000
#detected_icebergs = detected_icebergs[(detected_icebergs['area_km2'] >= min_size) & (detected_icebergs['area_km2'] < max_size)].reset_index()
detected_icebergs = detected_icebergs[(detected_icebergs['area_km2'] >= min_size) & (detected_icebergs['area_km2'] < max_size)]

#detected_icebergs = detected_icebergs.sort_values(by=['date', 'area_km2'], ascending=[True, False])

detected_icebergs.head()

## General informations

In [ ]:
num_ices = len(detected_icebergs['date'])
print('Total of icebergs detections: ', n_total_detections)
print('Total selected by size: ', num_ices)
print('Bigger area: ', detected_icebergs['area_km2'].max())
print('Smaller area: ', detected_icebergs['area_km2'].min())
print('========================================')

detected_icebergs.info()

In [ ]:
fig = plt.figure(figsize=(10,8))

#plt.figure(figsize=(10,8))
data = np.asarray(detected_icebergs['area_km2'].values)
n, bins, _ = plt.hist(data, bins=40, density=True, edgecolor='black', linewidth=1.2)

plt.vlines(np.nanmean(data), ymin=0, ymax=np.max(n), color='red', label='Average')

#plt.title("histogram")
plt.ylabel('Probability')
plt.xlabel('Area km$^{2}$')
plt.yscale('log')
plt.legend()
#plt.savefig('Area_distribution.jpg', dpi=300)

## Auxiliary heuristics

In [ ]:
def check_heuristics(delta_km, delta_days, area, speed, pairs=False):
    
    speed_tsh = 0.
    days_tsh = 0.
    
    if 0. < area <= 1.:
        radius = 9. * delta_days
        speed_tsh = 6.5 * 2
        days_tsh = 25
        
    if 1. < area <= 10.:
        radius = 6.5 * delta_days #* 2 7.5
        speed_tsh = 6.5 * 2
        days_tsh = 40
        
    if 10. < area <= 100.:
        radius = 4 * delta_days #* 2 5.5
        speed_tsh = 4. * 2
        days_tsh = 60
        
    if 100. < area <= 1000.:
        radius = 2.3 * delta_days * 1.5
        speed_tsh = 2.3 * 2
        days_tsh = 70 if area < 500 else 120
        
    if area > 1000.:
        radius = 2.5 * delta_days * 2 # 2.5
        speed_tsh = 2.32 * 2
        days_tsh = 90 if area < 1500 else 180
        
    #is_dist_max = True if delta_km < max_dist else False #550
    is_dist_max = True
    is_radius = True if delta_km <= radius else False
    is_speed = True if speed <= speed_tsh else False
    is_days = True if delta_days <= days_tsh else False
    
    return [is_dist_max, is_radius, is_speed, is_days]

In [ ]:
def moving_average(x, w):
    return (np.convolve(x, np.ones(w), 'valid') / w) ##.astype(int)

## Clean possible iceberg

In [ ]:
def clean_track(iceberg_to_clean, berg_individual_id, main_bar, smooth_len):

    save_base = iceberg_to_clean.copy()
    
    total_samples = len(iceberg_to_clean.index)
    
    last_id_valid = iceberg_to_clean.first_valid_index()
    last_id_valid_check = -1
    
    indexes_used = []
    
    while len(iceberg_to_clean.index) > 1:
        
        clear_output(wait=True)
        bar_length=20
        progress = float(total_samples-len(iceberg_to_clean.index))/(total_samples)
        block = int(round(bar_length * progress))
        progressbarmain = "Global progress: [{0}] {1:.1f}%".format( "#" * int(main_bar[0]) + "-" * (bar_length - int(main_bar[0])), main_bar[1] * 100.)
        progressbar = "Filtering progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100.)
        print(progressbarmain)
        print(progressbar, 'Total detections for current track: ', len(indexes_used))
        
        if last_id_valid != last_id_valid_check:
            
            area_base = iceberg_to_clean.loc[last_id_valid,'area_km2']
            
            iceberg_base_signature = moving_average(np.sort(np.asarray(json.loads(iceberg_to_clean.loc[last_id_valid,'shape']))).tolist(), smooth_len)

            iceberg_to_clean['jcd_to_base'] = iceberg_to_clean.apply(lambda row : measures.jaccard_similarity(iceberg_base_signature, moving_average(np.sort(np.asarray(json.loads(row['shape']))).tolist(),smooth_len)), axis = 1)
            iceberg_to_clean['k_s'] = iceberg_to_clean.apply(lambda row : 1 - stats.ks_2samp(iceberg_base_signature, moving_average(np.sort(np.asarray(json.loads(row['shape']))).tolist(),smooth_len))[0], axis = 1)
            #iceberg_to_clean['vonmisses'] = iceberg_to_clean.apply(lambda row : 1 - stats.cramervonmises_2samp(iceberg_base_signature, moving_average(np.sort(np.asarray(json.loads(row['shape']))).tolist(),smooth_len)).statistic, axis = 1)
            #iceberg_to_clean.loc[iceberg_to_clean['vonmisses'] < 0, 'vonmisses'] = 0
            
            iceberg_to_clean['sim_score'] = (iceberg_to_clean['jcd_to_base'] + iceberg_to_clean['k_s'])/2
            iceberg_to_clean = iceberg_to_clean.sort_values(by=['sim_score'], ascending=[False])
                
            if area_base < 1000:
                sim_tsh = 0.9
                sim_id = 1
            else:
                sim_tsh = 0.85
                sim_id = 1
        
        next_id = iceberg_to_clean.index[1]
        
        sims = [iceberg_to_clean.loc[next_id,'jcd_to_base'],
                iceberg_to_clean.loc[next_id,'k_s'],
                iceberg_to_clean.loc[next_id,'sim_score']]
             
        #if sim_pair_k_s > sim_tsh: #0.85
        if sims[sim_id] > sim_tsh:
                        
            indexes_used.append(last_id_valid)
            iceberg_to_clean.drop(last_id_valid, inplace=True)
            last_id_valid_check = last_id_valid
            last_id_valid = next_id
            
        else:
            
            iceberg_to_clean.drop(next_id, inplace=True)
            last_id_valid_check = last_id_valid
            
    
    if len(indexes_used) >= 5:
            
        berg_to_save = save_base.loc[indexes_used, :].sort_values(by=['date'], ascending=[True])
        berg_to_look = berg_to_save.copy()
        
        indexes_used_final = []
        while len(berg_to_look.index) > 0:
        
            indexes_used = berg_to_look.index.tolist()
            
            idx_start_look = berg_to_look.first_valid_index()
            idx_ant = idx_start_look
            row_ant = berg_to_look.loc[idx_ant, :]
            
            for index, row in berg_to_save.loc[idx_ant:, :].iterrows():

                droped = False
                if idx_ant != index:

                    dkm = round(geodist((row_ant['latitude'], row_ant['longitude']), (row['latitude'], row['longitude'])).km, 3)
                    ddays = abs((row['date'] - row_ant['date']).days)

                    ddays = 1. if ddays == 0. else ddays
                    dspeed = round(dkm/ddays, 3)

                    area_change = abs(row['area_km2'] - row_ant['area_km2'])
                    area_dst = area_change / row_ant['area_km2']

                    heuristics = check_heuristics(dkm, ddays, row_ant['area_km2'], dspeed, pairs=True)

                    #if not all(heuristics):
                    if not heuristics[1] or not heuristics[3] or not area_dst < 0.21 :
                        droped = True
                        indexes_used.pop(indexes_used.index(index))

                if not droped: 
                    row_ant = row
                    idx_ant = index
                    
            if len(indexes_used) > len(indexes_used_final):
                indexes_used_final = indexes_used
                
            berg_to_look.drop(idx_start_look, inplace=True)

            
        berg_to_look = berg_to_save.loc[indexes_used_final]
        if berg_to_look.shape[0] >= 3:
            berg_to_look.to_csv('./saidas/track_'+str(berg_individual_id)+'_'+str(indexes_used_final[0])+'.txt', mode = 'w', columns = ['date','latitude','longitude','majoraxis_km', 'area_km2'], sep=' ', index=False)
            
    else:
        indexes_used_final = indexes_used
        
    return indexes_used_final

## Main Loop tracking procedure

In [ ]:
t0 = time.process_time()

iceberg_control = detected_icebergs.copy().sort_values(by='date', ascending=True)

idx = iceberg_control.first_valid_index()

#delta_years_ths = 10
#smooth_len = 3

while len(iceberg_control.index) > 0:

    progressbar = 0
    indexes = iceberg_control.index
    idx = iceberg_control.first_valid_index()
    
    gc.collect()
    
    if idx in indexes:
        
        clear_output(wait=True)
        bar_length=20
        progress = float(num_ices-len(indexes))/(num_ices)
        block = int(round(bar_length * progress))
        progressbar = "Global progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100.)
        print(progressbar)
        
        main_bar = [block, progress]
        
        iceberg_base = iceberg_control.loc[idx, :]

        date_iceberg_base = iceberg_base['date']
        area_iceberg_base = iceberg_base['area_km2']
        
        if area_iceberg_base > 50 :
            smooth_len = 3
        else:
            smooth_len = 2
            
        if area_iceberg_base > 100:
            delta_years_ths = 10
        else:
            delta_years_ths = 5
        
        iceberg_base_morpho_signature = moving_average(np.sort(np.asarray(json.loads(iceberg_base['shape']))).astype(int).tolist(), smooth_len)
        
        c2 = iceberg_control['longitude'] > -65 ## APENAS PARA WEDDELL

        possible_icebergs_df = iceberg_control.loc[c2]
        possible_icebergs_df = possible_icebergs_df[((possible_icebergs_df['date'] - iceberg_base['date']).dt.days)/365 <= delta_years_ths]
        
        if not possible_icebergs_df.empty:
        
            possible_icebergs_df['jcd_to_base'] = possible_icebergs_df.apply(lambda row : measures.jaccard_similarity(iceberg_base_morpho_signature, moving_average(np.sort(np.asarray(json.loads(row['shape']))).tolist(),smooth_len)), axis = 1)
            possible_icebergs_df['k_s'] = possible_icebergs_df.apply(lambda row : 1 - stats.ks_2samp(iceberg_base_morpho_signature, moving_average(np.sort(np.asarray(json.loads(row['shape']))).tolist(),smooth_len))[0], axis = 1)
            #possible_icebergs_df['vonmisses'] = possible_icebergs_df.apply(lambda row : 1 - stats.cramervonmises_2samp(iceberg_base_morpho_signature, moving_average(np.sort(np.asarray(json.loads(row['shape']))).tolist(),smooth_len)).statistic, axis = 1)
            #possible_icebergs_df.loc[possible_icebergs_df['vonmisses'] < 0, 'vonmisses'] = 0
            
            possible_icebergs_df['sim_score'] = (possible_icebergs_df['jcd_to_base'] + possible_icebergs_df['k_s'])/2

            possible_individual_iceberg_df = possible_icebergs_df[(possible_icebergs_df['sim_score'] > 0.5)].sort_values(by='sim_score', ascending=False) #0.5
            
            if not len(possible_individual_iceberg_df['jcd_to_base']) == 0:

                index_to_drop = clean_track(possible_individual_iceberg_df, idx, main_bar, smooth_len)
                
                if index_to_drop == []:
                    index_to_drop = idx

            else:
                index_to_drop = idx

            iceberg_control.drop(index_to_drop, inplace=True)
        
        else:
            iceberg_control.drop(indexes, inplace=True)            
    
clear_output(wait=True)
bar_length=20
progress = float(num_ices)/(num_ices)
block = int(round(bar_length * progress))
progressbar = "Global progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100.)
print(progressbar)

t1 = (time.process_time() - t0) / 60.
print("Total time elapsed: ", round(t1, 3), 'minutes') # CPU seconds elapsed (floating point)